## Prototype use case : Safe night travel for women in New York (borough wise)

### Part 1
### Intialising the Google Maps API to use it's functionalities in our use case

In [1]:
#Reading the gmaps api key into the project and saving it in a variable
with open('gmaps_apikey.txt') as f:
    api_key = f.readline()
    f.close

In [2]:
#Loading the gmaps API to use further
import gmaps
gmaps.configure(api_key='...')

In [6]:
#Creating a simple map of New york for testing the API
ny_coordinates = (40.7128, -74.0060)
gmaps.figure(center=ny_coordinates, zoom_level=14)

Figure(layout=FigureLayout(height='420px'))

### Part 2

### For this particular use case prototype, the features used would be :
#### 1. Population density in each Borough 
#### 2. The count of crime related complaints in each borough 
#### 3. The count of street lights in each borough

### Loading the data, data cleaning and feature engineering

In [7]:
#Loading the relevant datasets for creating a layer on top of gmaps for our use case

import pandas as pd
import numpy as np

#Population density district wise in New York
population_density = pd.read_csv("/Users/sukanyak/Downloads/New_York_City_Population_By_Community_Districts.csv")

#Crime rate based on complaints registered area wise in New York
crime_rate = pd.read_csv("/Users/sukanyak/Downloads/NYPD_Complaint_Data_Historic.csv")

#Utility poles, street lights etc installed in areas of New York
utility = pd.read_csv("/Users/sukanyak/Downloads/Mobile_Telecommunication_Franchise_Poletop_Installation_Locations.csv")

/Users/sukanyak/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
#Checking the data schema to remedy and collate into one unified data set
population_density.head(10)
crime_rate.head(10)
utility.head(10)

,Id,X Coord.,Y Coord.,Latitude,Longitude,Zone,Franchisee,On street,Cross street 1,Cross street 2,Borough,Pole type,BID Advisory,Historic Advisory,NYSDOT Advisory,Park Advisory,Port Auth Advisory,School Advisory,Submission date,Equipment Installed? (Yes/No)
0,2040,990378,221185,40.773776,-73.977875,A,"Extenet Systems, INC. II",CENTRAL PK W,W 68 ST,W 69 ST,Manhattan,Streetlight Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
1,2041,990920,222025,40.776081,-73.975917,A,"Extenet Systems, INC. II",CENTRAL PK W,TERRACE DR,W 71 ST,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
2,2042,991563,223321,40.779638,-73.973594,A,"Extenet Systems, INC. II",Central Park West,77th Street,81st Street,Manhattan,Streetlight Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
3,2043,992114,224172,40.781973,-73.971604,A,"Extenet Systems, INC. II",CENTRAL PK W,TRANSVERSE RD 2,W 82 St,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
4,2044,992730,225433,40.785434,-73.969378,A,"Extenet Systems, INC. II",Central Park West,86th Street,87th Street,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
5,2047,993249,220420,40.771674,-73.967511,A,"Extenet Systems, INC. II",5th Avenue,71st Street,70th Street,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
6,2048,992744,219509,40.769174,-73.969335,A,"Extenet Systems, INC. II",5th Avenue,67th Street,66th Street,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
7,2049,992493,219051,40.767917,-73.970242,A,"Extenet Systems, INC. II",5th Avenue,65th Street,64th Street,Manhattan,Mixed Traffic And Street Pole,NaN,This pole is located within the following hist...,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
8,2064,987699,221769,40.775380,-73.987547,A,"Extenet Systems, INC. II",West End Avenue,65th Street,66th Street,Manhattan,Mixed Traffic And Street Pole,NaN,NaN,NaN,NaN,NaN,NaN,Jun-16-2005,Yes
9,2066,986690,219928,40.770328,-73.991191,A,"Extenet Systems, INC. II",11th Avenue,57th Street,58th Street,Manhattan,Mixed Traffic And Street Pole,NaN,NaN,NaN,NaN,NaN,This pole is located within 20 feet of the fol...,Jun-16-2005,Yes


In [72]:
#Taking the average population density in each borough
density_res = population_density.groupby(['Borough']).mean()

#Selecting only the relevant columns required for our solution. Here, we have the data for 2010 which is the latest survey
density_res = density_res[['2010 Population']]

In [71]:
#Taking the total count of crime complaints in each borough
crime_res = crime_rate.groupby(['PATROL_BORO']).count()

#Selecting only the relevant columns
crime_res = crime_res.iloc[:,0:1]

In [75]:
#Taking the total count of street lights in each borough
utility_res = utility.groupby(['Borough']).count()

#Selecting only the relevant columns
utility_res = utility_res.iloc[:,0:1]

In [104]:
#Checking the final 3 dataframes before combining into one unified dataset
density_res
crime_res
utility_res

,Borough,light_count
0,Bronx,1415
1,Brooklyn,1117
2,Manhattan,2323
3,Queens,1266
4,Staten Island,491


For lack of time, I have exported the final datasets into excel as csv for additional formatting and then imported them back.
Now the 3 files of the 3 features are ready to be merged into 1 unified dataset, where we will calculate a score to identify an area as safe/unsafe based on these features and then create safety hotspots on gmaps as an additional layer.

**The latitude, longitude and area data is input manually here since only 5 boroughs

In [112]:
#Unified_df = pd.concat([density_res,crime_res,utility_res], axis = 1, join='inner')

In [141]:
#Unified_df = pd.read_csv("/Users/sukanyak/Downloads/unified.csv")

In [146]:
Unified_df

,Borough,Latitude,Longitude,Area,avg_pop_density,complaint_count,light_count,avg_complaint_per_person,avg_light_sqkm,Safety_wt
0,Bronx,40.84,-73.86,110,115058.0833,1308131,1415,11.369310,12.863636,1.131435
1,Brooklyn,40.68,-73.94,180,139144.9444,1799669,1117,12.933772,6.205556,0.479795
2,Manhattan,40.78,-73.97,59,131448.7500,1442730,2323,10.975608,39.372881,3.587307
3,Queens,40.73,-73.79,280,160072.9286,1194357,1266,7.461330,4.521429,0.605982
4,Staten Island,40.58,-74.15,152,155989.3333,286260,491,1.835125,3.230263,1.760241


In [143]:
#Creating a new column for average complaints in each borough for each person. This would be useful to the final score
Unified_df['avg_complaint_per_person'] = Unified_df['complaint_count']/Unified_df['avg_pop_density']

In [144]:
#Calculating avg number of street light per sq km of each borough
Unified_df['avg_light_sqkm'] = Unified_df['light_count']/Unified_df['Area']

## Assumption and Approach for the Use Case:
### Here, we are going to calculate a cumulative weight based on the features that we have used to signify a borough as safe or unsafe.
### For calculating the weights, our basic assumption is that, in each borough,
### 1. safety is inversely proportional to the average complaints per person
### 2. safety is directly proportional to the average number of street lights present per square kilometer

In [145]:
#Calculating final weights to to give a safety score
Unified_df['Safety_wt'] = Unified_df['avg_light_sqkm']/Unified_df['avg_complaint_per_person']

# Rendering the final map with the added layer showing hotspots for safety

In [158]:
figure = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(Unified_df[['Latitude', 'Longitude']], max_intensity= 2, weights=Unified_df['Safety_wt'], opacity=1)
figure.add_layer(heatmap_layer)

figure

Figure(layout=FigureLayout(height='420px'))